In [ ]:
%matplotlib inline 
# this just tells the notebook to display images in the cells, rather than send it to a new window
try:
    # library for nicer visualizations
    import seaborn
    seaborn.set_context('poster')
except ImportError:
    pass

# Pandas and Databases

## *'If you torture the data long enough, it will confess.'*
*– Master Turtle*

Databases are collections of **tables**, organized into **columns** and **rows**, that allow us to store, select, and relate data to each other. The most common language is SQL, which comes in various forms.

`pandas` is a library (i.e., a code collection) of useful functions and objects to work with data. It can read from a variety of formats (CSV, Excel, ...), including SQL databases, and has a load of nifty tools to analyze and visualize the data.

# Databases

## Selecting data

In SQL, keywords are typically written in CAPS, as to distinguish them from variable names. The general command to get data from a DB is

```sql
SELECT <columns> FROM <table>
```

`<columns>` is a comma separated list of the column names we want. If we want **all** columns, we can use the Kleene star `*`.

### DISTINCT

If we want unique entries, we can use the `DISTINCT` keyword:
```sql
SELECT DISTINCT <columns> FROM <table>
```

This automatically gets rid of duplicates.

### WHERE

If we want only entries that fulfill a certain condition, we can use the `WHERE` keyword:
```sql
SELECT <columns> FROM <table> WHERE <condition>
```

`<condition>` is similar to a boolean statement in Python, but you don't need double equals for comparison. 

```sql
SELECT * FROM tweets WHERE followers > 100
```

You can also combine several conditions with `AND` and `OR`.

```sql
SELECT * FROM tweets WHERE followers > 100 AND is_retweet = 'False'
```



### LIMIT

Databases can be massive (that's kind of their point), so when you only want a bit to explore first, use the `LIMIT` keyword:
```sql
SELECT <columns> FROM <table> LIMIT <N>
```

`<N>` is the maximum number of entries you want

## Activity

Suppose you have a table called `movies` with the following columns:

<table>
    <tr>
        <th>name</th>
        <th>director</th>
        <th>year</th>
        <th>cost</th> 
        <th>gain</th>
    </tr>
</table>

How would you select
* the unique names of all the directors?
* all rows with films made in 1992?
* all films made before 2010 that cost under 10M?
* the top 10 films made in 2001 that made over 10M?

## Databases in Python

In order to use SQL in Python, we need a library called `sqlite3`.

In [ ]:
import sqlite3 # this library allows us to connect to a database

First, we need to establish a connection to the database. We use the `connect()` function:

In [ ]:
con = sqlite3.connect('../data/example.db')

Databases can contain several **tables**, so before we proceed, it's a good idea to check how many we have and how they are called. To do this, we first need a **cursor** (think of it as an iterator over the table), and then execute an **SQL command**:

In [ ]:
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

# `pandas`

In [ ]:
import pandas as pd # import pandas, but give it a shorthand name

`pandas` let's us read a table directly into Python. We just need a connection and a SQL command.

In [ ]:
df = pd.read_sql("SELECT * FROM reviews", con=con)

The result of this process is a new object, called a **`DataFrame`**. You can think of it as a table on steroids.

`pandas` can read all kinds of *X*-separated files (using `read_csv()`), as well as Excel (`read_excel`).

By default, `read_csv()` uses commas `,` as separators, but we can tell it to use any other separator, for example tabs `\t` instead.

Now, we can look at the data. The file is rather big, so we will only look at the top 5 lines. We use the `head()` function for it.

The top row contains the column names.

In [ ]:
df.head(2)

If you want to look at more lines, you need to give that number to the function as argument.

We can also look at the bottom rows, with `tail()`

## Activity: Loading data

* create a new variable, `scores`, and read in the distinct scores from the SQL table
* look at the last 3 lines of `scores`

In [ ]:
# your code here


# Indexing

### Columns

DataFrames, as well as databases, are organized in columns with names. To get an overview, use the `columns` attribute.

In [ ]:
df.columns

To see just one column, access it with the same square bracket notation as for lists `[]`. However, rather than a number, use the column name string.

Alternatively, you can treat the name like an attribute of the `DataFrame`, and use a dot operator.

In [ ]:
# Dictionary-style indexing
texts1 = df['text']

# Alternative syntax
texts2 = df.text

# check whether the Series are the same
print(texts1 is texts2)

The column returned from such an indexing is called a `Series` object.

To select several columns, we need to use the first method, and give it a list of column names. This returns a new `DataFrame` object.

In [ ]:
print(type(df.text))
print(type(df[['text', 'score']]))

#### Series functions

There are a lot of functions to analyze a `Series` object. One of the most useful is `describe()`, which gives us some descriptive statistics. Some of the ones used here, like `count()`, `mean()`, `max()`, and `min()` can be called by themselves.

In [ ]:
df.score.describe()

## Acticvity

* what are the mean and median of the `age` column?
* what do you get when you use describe on a column with strings, like `text` or `gender`?

In [ ]:
# your code here


### Rows

Retrieving a row is also possible, but uses a different method, `iloc` (for **i**nteger **loc**ation), and the index of the row.

In [ ]:
df.iloc[2]

As with lists, you can also use slices, and even lists of integers:

In [ ]:
print(df.text.iloc[2:4])
print(df.text.iloc[[1,1,5,11]])

## Adding new columns

Let's create a new column `ratio`, i.e., the amount of score per year of age, based on the existing columns `score` and `age`.

In [ ]:
df['ratio'] = (df.score / df.age)
df.ratio.describe()

## Getting rid of outliers

## Masks

Say we want to remove all ratios below a certain threshold.

In dataframes, we can select all the data that matches a certain condition, similar to an SQL table. We can do that with a simple boolean statement. The result is a `Series` object with boolean values, also called a **mask**:

In [ ]:
too_low = df.ratio < 0.1
print(too_low.head())
print(type(too_low))

## Activity

* add another, boolean, column to `df`, that signals whether the entry contains the word `price`. Call the new column `has_price`

In [ ]:
# your code here


We can combine several conditions with the `&` or `|` operators.

In [ ]:
over50 = df.age >= 0

print(len(df), len(df[too_low | over50]), len(df[too_low & over50]))

If we apply this mask to our DataFrame, we get *only* the rows where the condition is `True`

In [ ]:
df[too_low & over50].iloc[[1, 10]]

The value for `ratio`, `NaN`, stands for "**N**ot **a** **N**umber". Luckily, we can replace that value with `fillna()`

In [ ]:
df_no_nan = df.fillna('THIS WAS NAN')
df_no_nan

## apply()

Let's also get rid of the `inf` values. We write a simple function that sets them to `0` and `apply` that function to our `Series`.

In [ ]:
def f(x):
    if x > 65:
        return 'retired'
    else:
        return 'not_retired'
    
df_no_nan['status'] = df_no_nan.age.copy().apply(f)
df_no_nan.head()

## Visualization

### Histograms

`pandas` `DataFrame`s have built-in visualization methods under the property `plot`. To get a histogram of the ratios, we can call `hist()`

In [ ]:
df_no_nan.age.plot.hist()

In order to get a different representation, let's define the size of each `bin`. Adding `;` at the end of the line prevents the output of the `<matplotlib.axes._subplots.AxesSubplot at 0x118903320>` text.

In [ ]:
df_no_nan.age.hist(bins=50);

We can even separate the plots by another column, for example `gender`, using the keyword `by`.

In [ ]:
df_no_nan.age.hist(by=df_no_nan.gender, bins=20);

Looks similar, but at different scales. To make the y-axis equivalent, use `sharey`

In [ ]:
df_no_nan.age.hist(by=df_no_nan.gender, bins=20, sharey=True);

### Scatterplotting

If we are interested in more than one column, we might want to use the general `plot` method, and the `scatter` function. We need to define the `x` and `y` dimensions by giving a column name from our `DataFrame`.

In [ ]:
df_no_nan.plot.scatter(x='age', y ='ratio', s=100, alpha=0.5);

To separate the two categories, you need to 
* make the first one a variable
* pass the variable to the second via `ax`
* set different colors

In [ ]:
axis = df_no_nan[df_no_nan.category == 'Gambling'].plot.scatter(x='age', y ='score', color='teal', s=100, alpha=0.5);
df_no_nan[df_no_nan.category == 'Sport'].plot.scatter(x='age', y ='score', color='darkred', ax=axis, s=100, alpha=0.5);

For many more plotting options, see [https://pandas.pydata.org/pandas-docs/stable/visualization.html](https://pandas.pydata.org/pandas-docs/stable/visualization.html)

For more on visualizations in general, see [http://serialmentor.com/dataviz/](http://serialmentor.com/dataviz/)

## Correlations

The scatter plot suggested that there is some correlation in the data. Let's check. Luckily, there's a function for that: `corr()`.

In [ ]:
df_no_nan[['age', 'score', 'ratio']].corr()

## Views

We can subselect a number of columns via slicing (a *view*) and put them in a new `DataFrame`by using `copy()`.

In [ ]:
new_view = df_no_nan[['age', 'score', 'category', 'text']].copy()
new_view.head()

## Grouping

We can group our data by columns. This is useful for example in visualization. Rather than showing two plots as before, we can overlay them (we use `alpha` to set the transparency):

In [ ]:
new_view[:1000].groupby('category').score.hist(bins=20, alpha=0.5);

We can also get aggregate statistics for the groups. We use `groupby()`, and then specify how to aggregate the data. Possible aggregators are
* `sum()`
* `count()`
* `min()`
* `max()`
* `mean()`
* `median()`

... and many more. See [https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/](https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/) for more examples